In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
import keras
import pickle
import fasttext
from keras.callbacks import LearningRateScheduler
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.layers import *
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard, ReduceLROnPlateau
from nlpaug.util.file.download import DownloadUtil

import nlpaug.augmenter.word as naw
from tqdm import tqdm
# pd.options.mode.chained_assignment = None

In [2]:
data = pd.read_csv("final_data.csv")
data.drop(['Unnamed: 0'],axis=1,inplace=True)

In [3]:
data.head(5)

,corupted_text,normal_text_input,normal_text_output
0,"U wan me to ""chop"" seat 4 u nt?",<start> Do you want me to reserve seat for you...,Do you want me to reserve seat for you or not?...
1,Yup. U reaching. We order some durian pastry a...,<start> Yeap. You reaching? We ordered some Du...,Yeap. You reaching? We ordered some Durian pas...
2,They become more ex oredi... Mine is like 25.....,<start> They become more expensive already. Mi...,They become more expensive already. Mine is li...
3,I'm thai. what do u do?,<start> I'm Thai. What do you do?,I'm Thai. What do you do? <end>
4,Hi! How did your week go? Haven heard from you...,<start> Hi! How did your week go? Haven't hear...,Hi! How did your week go? Haven't heard from y...


In [ ]:
path = "pickles"
model = tf.saved_model.load(path)

In [ ]:
# loading the dumped file
filename = "model_and_tokenizers/tokenizer_source.pkl"
with open(filename, 'rb') as file:
  tokenizer_source = pickle.load(file)
filename = "model_and_tokenizers/tokenizer_target.pkl"
with open(filename, 'rb') as file:
  tokenizer_target = pickle.load(file)

In [1]:
max_len=39
max_len_dec=44

In [42]:
def predict(input_sentence):

  input_sequence=tokenizer_source.texts_to_sequences([input_sentence])
  inputs=pad_sequences(input_sequence, maxlen=max_len, padding='post')
  inputs=tf.convert_to_tensor(inputs)
  result=''
  units=128
  hidden=[tf.zeros((1,units))]
  encoder_output,hidden_state,cell_state=model.encoder(inputs)
  dec_hidden=hidden_state
  dec_input=tf.expand_dims([tokenizer_target.word_index['<start>']],0)
  for t in range(40):
      predictions,dec_hidden,cell_state,attention_weights,context_vector=model.decoder.onestepdecoder((dec_input,encoder_output,dec_hidden,cell_state))

      predicted_id=tf.argmax(predictions[0]).numpy()
      result+=tokenizer_target.index_word[predicted_id]+' '
      if tokenizer_target.word_index['<end>']==predicted_id:
          return result
      dec_input= tf.expand_dims([predicted_id],0)
  return result

In [45]:
for i in range(0,10):
    input_sentence=test["corupted_text"].iloc[i]
    print('Input:',input_sentence)
    print('Prediction:',predict(input_sentence))
    print('Actual:',test["normal_text_output"].iloc[i])
    print('*'*100)

Input: So what new insights hav u gained from my ans to ur qn?
Prediction: So what new insights have you gained from my answers to your question <end> 
Actual: So what new insights have you gained from my answers to your question? <end> <end>
****************************************************************************************************
Input: By the way i'm malay hope you guys don't mind
Prediction: By the way I'm Malay hope you guys don't mind <end> 
Actual: By the way, I'm Malay, hope you guys don't mind. <end>
****************************************************************************************************
Input: Goto 2 malayu room lah! U can find sme cute gals!!!
Prediction: Go to Malayu room You can find some cute girls <end> 
Actual: Go to Malayu room! You can find some cute girls! <end>
****************************************************************************************************
Input: Oh... Lk tt ah... Wat kind of jobs u wan... Waitress or office, i help u look 